In [252]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import poisson

In [253]:
def generate_completion_times(n=2000):
    completion_normal_df = np.random.normal(8,8,n)
    return np.round(np.add(completion_normal_df, abs(np.min(completion_normal_df)))).astype('int')

#generate_completion_times(20)

In [254]:
a = np.array([[1,2], [2,3], [5]])
np.hstack(a)

<ipython-input-254-81ffb8360d36>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  a = np.array([[1,2], [2,3], [5]])


array([1, 2, 2, 3, 5])

In [344]:
'''
@input: n - initial number of events
@return: final number (>= n ) of generated events (flattened list)
'''
def generate_events(n=2000):
    events = np.empty_like((), dtype=object, shape=(n,))
    # These lambdas were extrapolated from all_tickets.csv
    lambdas = [1,4,3,18]
    
    # Normalize the lambdas
    mu = lambdas / np.sum(lambdas)
    
    # Get the prob mass func
    pmf = poisson.pmf(k=1, mu=mu)

    np.random.seed(12345)
    
    completion_times = generate_completion_times(n)
    
    for i in range(n):
        if type(events[i]) != list:
            events[i] = []
        
        current_event_priority = np.where(lambdas == np.random.choice(lambdas, size=1, p=mu))[0][0]
        current_event_status = 'pending'
        events[i].append((i+1,current_event_priority,current_event_status))
        
        # Completion times of any ticket is drawn from a normal distribution
        future_event_hour = i + completion_times[i]
        
        if future_event_hour < n:
            if type(events[future_event_hour]) != list:
                events[future_event_hour] = []
            
            events[future_event_hour].append((i+1,current_event_priority, 'completed'))
            
    return np.concatenate(events)

generate_events(20)

array([['1', '3', 'pending'],
       ['2', '3', 'pending'],
       ['3', '3', 'pending'],
       ['4', '1', 'pending'],
       ['5', '0', 'pending'],
       ['6', '3', 'pending'],
       ['7', '3', 'pending'],
       ['8', '0', 'pending'],
       ['9', '3', 'pending'],
       ['10', '3', 'pending'],
       ['11', '3', 'pending'],
       ['12', '1', 'pending'],
       ['13', '3', 'pending'],
       ['14', '3', 'pending'],
       ['1', '3', 'completed'],
       ['3', '3', 'completed'],
       ['15', '3', 'pending'],
       ['4', '1', 'completed'],
       ['16', '3', 'pending'],
       ['17', '3', 'pending'],
       ['17', '3', 'completed'],
       ['12', '1', 'completed'],
       ['18', '1', 'pending'],
       ['19', '2', 'pending'],
       ['20', '2', 'pending']], dtype='<U21')

In [345]:
max_pending_assignments = 50
def generate_states(max_states=50):
    a = np.arange(max_states)
    arrays = 4 * [a]
    grid = np.meshgrid(*arrays)        
    coord_list = [entry.ravel() for entry in grid]
    points = np.vstack(coord_list).T
    return points

In [346]:
def epsilon_greedy(val, max_val):
    return 0.5 / (1 + (np.exp((10 * (val - 0.4*max_val))/max_val)))

epsilon_greedy(1, 2000)

0.4909626316679083

In [362]:
K = 2000   #episodes
n = np.array(['A','B','C','D','E','F','G','H','I','J'])     #users
states = np.array([generate_states(10)] * K)
events = generate_events(250*8)
actions = np.empty_like('', dtype=int, shape=(K,len(events)))
rewards = np.zeros((K,len(events)))

Q = np.zeros((states.shape[1],len(n)))
Q.shape

(10000, 10)

In [363]:
def get_state_index(k, input_state):
    return np.where((states[k,:,0] == input_state[0]) & \
             (states[k,:,1] == input_state[1]) & \
             (states[k,:,2] == input_state[2]) & 
             (states[k,:,3] == input_state[3]))[0][0]
#get_state_index(5, [1,2,2,3])

In [364]:
'''
State transition function
@input: event[t], S[t]
@returns: S[t+1] = F(event[t], S[t])
'''
def process_event(current_event, current_state):
    next_state = np.copy(current_state)        ## To be modified below
    incoming_priority = int(current_event[1])
    if current_event[2] == 'pending':
        next_state[incoming_priority] = min(next_state[incoming_priority] + 1, 9)
    elif current_event[2] == 'completed':
        next_state[incoming_priority] = max(next_state[incoming_priority] - 1, 0)
    else:
        print ("Priority value " + current_event[2] + " not recognized, no change in state")
    
    return next_state

In [366]:
# index0 = user, index1 = total_pending_jobs, index2 = cumulative_estimated_time
observable_environment = np.array([np.zeros((2,))] * len(n))

In [367]:
'''
objective_function = -0.5 * (total_expected_times) -0.5 * (variance_pending_jobs)
'''
def observed_reward(current_event, action_taken_idx):
    _, priority, status = current_event
    if status == 'pending':
        observable_environment[action_taken_idx][0] += 1
    elif status == 'completed':
        observable_environment[action_taken_idx][0] -= 1
    
    observable_environment[action_taken_idx][1] = observable_environment[action_taken_idx][0] * \
                                                                generate_completion_times(1)
    
    expected_reward = -0.5 * observable_environment[:,1].sum() - 0.5 * observable_environment[:,0].var()
    
    return expected_reward

In [368]:
learning_rate = 0.6
gamma = 0.5
runtime_states = []

In [369]:
#x = []
#y = []
#y.append([0,0,0,0])
#y.append([0,1,0,0])
#x.append(y)

In [391]:
for k in range(K):
    tmp_state_list = []
    epsilon = epsilon_greedy(k, K)
    for t in range(len(events)):
        tmp_state_list.append(states[k,t])
        current_state_index = get_state_index(k, states[k,t])
        p = np.random.random()
        if p < epsilon:
            actions[k,t] = np.random.choice(np.arange(len(n)))
        else:
            actions[k,t] = np.argmax(Q[current_state_index])
            
        # Next state
        next_state = process_event(events[t], states[k,t])
        tmp_state_list.append(next_state)
        #print (k, next_state, actions[k,t])
        next_state_index = get_state_index(k, next_state)
            
        #Reward for current action
        rewards[k,t] = observed_reward(events[t], actions[k,t])
        
        #Recompute Q
        effective_reward = rewards[k,t] + (gamma * np.max(Q[next_state_index, :])) - Q[current_state_index, actions[k,t]]
        Q[current_state_index, actions[k,t]] += learning_rate * effective_reward
        
    print ("Episode " + str(k+1) + " completed") 

Episode 1 completed
Episode 2 completed
Episode 3 completed
Episode 4 completed
Episode 5 completed
Episode 6 completed
Episode 7 completed
Episode 8 completed
Episode 9 completed
Episode 10 completed
Episode 11 completed
Episode 12 completed
Episode 13 completed
Episode 14 completed
Episode 15 completed
Episode 16 completed
Episode 17 completed
Episode 18 completed
Episode 19 completed
Episode 20 completed
Episode 21 completed
Episode 22 completed
Episode 23 completed
Episode 24 completed
Episode 25 completed
Episode 26 completed
Episode 27 completed
Episode 28 completed
Episode 29 completed
Episode 30 completed
Episode 31 completed
Episode 32 completed
Episode 33 completed
Episode 34 completed
Episode 35 completed
Episode 36 completed
Episode 37 completed
Episode 38 completed
Episode 39 completed
Episode 40 completed
Episode 41 completed
Episode 42 completed
Episode 43 completed
Episode 44 completed
Episode 45 completed
Episode 46 completed
Episode 47 completed
Episode 48 completed
E

Episode 379 completed
Episode 380 completed
Episode 381 completed
Episode 382 completed
Episode 383 completed
Episode 384 completed
Episode 385 completed
Episode 386 completed
Episode 387 completed
Episode 388 completed
Episode 389 completed
Episode 390 completed
Episode 391 completed
Episode 392 completed
Episode 393 completed
Episode 394 completed
Episode 395 completed
Episode 396 completed
Episode 397 completed
Episode 398 completed
Episode 399 completed
Episode 400 completed
Episode 401 completed
Episode 402 completed
Episode 403 completed
Episode 404 completed
Episode 405 completed
Episode 406 completed
Episode 407 completed
Episode 408 completed
Episode 409 completed
Episode 410 completed
Episode 411 completed
Episode 412 completed
Episode 413 completed
Episode 414 completed
Episode 415 completed
Episode 416 completed
Episode 417 completed
Episode 418 completed
Episode 419 completed
Episode 420 completed
Episode 421 completed
Episode 422 completed
Episode 423 completed
Episode 42

Episode 752 completed
Episode 753 completed
Episode 754 completed
Episode 755 completed
Episode 756 completed
Episode 757 completed
Episode 758 completed
Episode 759 completed
Episode 760 completed
Episode 761 completed
Episode 762 completed
Episode 763 completed
Episode 764 completed
Episode 765 completed
Episode 766 completed
Episode 767 completed
Episode 768 completed
Episode 769 completed
Episode 770 completed
Episode 771 completed
Episode 772 completed
Episode 773 completed
Episode 774 completed
Episode 775 completed
Episode 776 completed
Episode 777 completed
Episode 778 completed
Episode 779 completed
Episode 780 completed
Episode 781 completed
Episode 782 completed
Episode 783 completed
Episode 784 completed
Episode 785 completed
Episode 786 completed
Episode 787 completed
Episode 788 completed
Episode 789 completed
Episode 790 completed
Episode 791 completed
Episode 792 completed
Episode 793 completed
Episode 794 completed
Episode 795 completed
Episode 796 completed
Episode 79

Episode 1119 completed
Episode 1120 completed
Episode 1121 completed
Episode 1122 completed
Episode 1123 completed
Episode 1124 completed
Episode 1125 completed
Episode 1126 completed
Episode 1127 completed
Episode 1128 completed
Episode 1129 completed
Episode 1130 completed
Episode 1131 completed
Episode 1132 completed
Episode 1133 completed
Episode 1134 completed
Episode 1135 completed
Episode 1136 completed
Episode 1137 completed
Episode 1138 completed
Episode 1139 completed
Episode 1140 completed
Episode 1141 completed
Episode 1142 completed
Episode 1143 completed
Episode 1144 completed
Episode 1145 completed
Episode 1146 completed
Episode 1147 completed
Episode 1148 completed
Episode 1149 completed
Episode 1150 completed
Episode 1151 completed
Episode 1152 completed
Episode 1153 completed
Episode 1154 completed
Episode 1155 completed
Episode 1156 completed
Episode 1157 completed
Episode 1158 completed
Episode 1159 completed
Episode 1160 completed
Episode 1161 completed
Episode 116

Episode 1476 completed
Episode 1477 completed
Episode 1478 completed
Episode 1479 completed
Episode 1480 completed
Episode 1481 completed
Episode 1482 completed
Episode 1483 completed
Episode 1484 completed
Episode 1485 completed
Episode 1486 completed
Episode 1487 completed
Episode 1488 completed
Episode 1489 completed
Episode 1490 completed
Episode 1491 completed
Episode 1492 completed
Episode 1493 completed
Episode 1494 completed
Episode 1495 completed
Episode 1496 completed
Episode 1497 completed
Episode 1498 completed
Episode 1499 completed
Episode 1500 completed
Episode 1501 completed
Episode 1502 completed
Episode 1503 completed
Episode 1504 completed
Episode 1505 completed
Episode 1506 completed
Episode 1507 completed
Episode 1508 completed
Episode 1509 completed
Episode 1510 completed
Episode 1511 completed
Episode 1512 completed
Episode 1513 completed
Episode 1514 completed
Episode 1515 completed
Episode 1516 completed
Episode 1517 completed
Episode 1518 completed
Episode 151

Episode 1833 completed
Episode 1834 completed
Episode 1835 completed
Episode 1836 completed
Episode 1837 completed
Episode 1838 completed
Episode 1839 completed
Episode 1840 completed
Episode 1841 completed
Episode 1842 completed
Episode 1843 completed
Episode 1844 completed
Episode 1845 completed
Episode 1846 completed
Episode 1847 completed
Episode 1848 completed
Episode 1849 completed
Episode 1850 completed
Episode 1851 completed
Episode 1852 completed
Episode 1853 completed
Episode 1854 completed
Episode 1855 completed
Episode 1856 completed
Episode 1857 completed
Episode 1858 completed
Episode 1859 completed
Episode 1860 completed
Episode 1861 completed
Episode 1862 completed
Episode 1863 completed
Episode 1864 completed
Episode 1865 completed
Episode 1866 completed
Episode 1867 completed
Episode 1868 completed
Episode 1869 completed
Episode 1870 completed
Episode 1871 completed
Episode 1872 completed
Episode 1873 completed
Episode 1874 completed
Episode 1875 completed
Episode 187

In [400]:
Q[3969]

array([-2139068.9941256 , -2146433.59423109, -2079462.46971376,
       -2075164.82924518, -2257003.08815144, -2016073.4874243 ,
       -2150588.99584313, -2075197.46915676, -2082391.97100388,
       -2248821.09468248])

In [401]:
observable_environment

array([[  6880.,  82560.],
       [  5635.,  84525.],
       [  5748.,      0.],
       [  7830., 289710.],
       [  4977.,  29862.],
       [  7585., 174455.],
       [  5771.,  86565.],
       [  8247.,  90717.],
       [  6453., 174231.],
       [  6455.,      0.]])